In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    n_estimators: int
    max_depth: int
    min_child_weight: int
    gamma: float
    subsample: float
    colsample_bytree: float
    reg_alpha: float
    reg_lambda: float
    target_column: str
    

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

[2025-02-21 23:34:35,042 : INFO : __init__ : Logger has been set up successfully!]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            learning_rate=params.learning_rate,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_child_weight=params.min_child_weight,
            gamma=params.gamma,
            subsample=params.subsample,
            colsample_bytree=params.colsample_bytree,
            reg_alpha=params.reg_alpha,
            reg_lambda=params.reg_lambda,
            target_column = schema.name
        )

        return model_trainer_config

In [7]:
from xgboost import XGBRegressor
from mlproject import logger
import pandas as pd
import joblib
import numpy as np

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Validate file paths
        if not os.path.exists(self.config.train_data_path):
            raise FileNotFoundError(f"Training data file not found at {self.config.train_data_path}")
        if not os.path.exists(self.config.test_data_path):
            raise FileNotFoundError(f"Testing data file not found at {self.config.test_data_path}")

        # Load the data
        train_data = np.load(self.config.train_data_path, allow_pickle=True)
        test_data = np.load(self.config.test_data_path, allow_pickle=True)

        logger.info(f"Loaded train data: type={type(train_data)}, shape={train_data.shape}")
        logger.info(f"Loaded test data: type={type(test_data)}, shape={test_data.shape}")

        # Split features and target
        train_x = train_data[:, :-1]  # All columns except the last one
        train_y = train_data[:, -1]   # Only the last column
        test_x = test_data[:, :-1]    # All columns except the last one
        test_y = test_data[:, -1]     # Only the last column

        logger.info(f"Training data shape: X={train_x.shape}, y={train_y.shape}")
        logger.info(f"Testing data shape: X={test_x.shape}, y={test_y.shape}")

        # Train the model
        logger.info("Initializing XGBRegressor...")
        regressor = XGBRegressor(
            learning_rate=self.config.learning_rate,
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            min_child_weight=self.config.min_child_weight,
            gamma=self.config.gamma,
            subsample=self.config.subsample,
            colsample_bytree=self.config.colsample_bytree,
            reg_alpha=self.config.reg_alpha,
            reg_lambda=self.config.reg_lambda,
            random_state=42
        )
        logger.info("Training the model...")
        regressor.fit(train_x, train_y)

        # Save the trained model
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(regressor, model_path)
        logger.info(f"Model saved successfully at {model_path}")

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-02-21 23:34:36,743 : INFO : common : yaml file: config\config.yaml loaded successfully]


[2025-02-21 23:34:36,748 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-02-21 23:34:36,752 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-02-21 23:34:36,754 : INFO : common : created directory at: artifacts]
[2025-02-21 23:34:36,755 : INFO : common : created directory at: artifacts/model_trainer]
[2025-02-21 23:34:36,998 : INFO : 1420427480 : Loaded train data: type=<class 'numpy.ndarray'>, shape=(225114, 37)]
[2025-02-21 23:34:37,002 : INFO : 1420427480 : Loaded test data: type=<class 'numpy.ndarray'>, shape=(75039, 37)]
[2025-02-21 23:34:37,002 : INFO : 1420427480 : Training data shape: X=(225114, 36), y=(225114,)]
[2025-02-21 23:34:37,002 : INFO : 1420427480 : Testing data shape: X=(75039, 36), y=(75039,)]
[2025-02-21 23:34:37,004 : INFO : 1420427480 : Initializing XGBRegressor...]
[2025-02-21 23:34:37,004 : INFO : 1420427480 : Training the model...]
[2025-02-21 23:34:50,161 : INFO : 1420427480 : Model saved successfully at artifacts/model_t